# Install the required dependencies

In [1]:
%pip install miditok
%pip install miditoolkit
%pip install pretty_midi
%pip install tqdm

     ---------------------------------------- 0.0/57.6 kB ? eta -:--:--
     --------------------- ------------------ 30.7/57.6 kB 1.4 MB/s eta 0:00:01
     ---------------------------------------- 57.6/57.6 kB 1.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/119.2 kB ? eta -:--:--
   ------------------------------ --------- 92.2/119.2 kB 1.7 MB/s eta 0:00:01
   ---------------------------------------- 119.2/119.2 kB 1.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/346.4 kB ? eta -:--:--
   ---------------------------------------  337.9/346.4 kB 7.0 MB/s eta 0:00:01
   ---------------------------------------- 346.4/346.4 kB 5.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ----- ---------------------------------- 0.3/2.0 MB 8.9 MB/s eta 0:00:01
   --------------------- ------------------ 1.0/2.0 MB 13.3 MB/s eta 0:00:01
   --------------------- ------------------ 1.0/2.0 MB 13.3 MB/s eta 0:00:01
   -----

# Import required packages and load the dataset

In [2]:
from pathlib import Path
from copy import deepcopy
from math import ceil

import pretty_midi
from miditoolkit import MidiFile
from tqdm import tqdm

from typing import List, Dict

### Define the parameters for chunking

In [5]:
MAX_NB_BAR = 8
MIN_NB_NOTES = 16
path = "test/prompt"

` The load_dataset function creates new directory and loads the midi paths`

In [6]:
def load_valid_midi_files(path: str) -> Dict[str, bool]:
    """
    Load and validate MIDI files from a specified path based on the number of notes,
    and return a dictionary mapping MIDI file paths to a boolean indicating validity.

    Parameters:
    - path (str): The path to the dataset directory.

    Returns:
    - Dict[str, bool]: A dictionary where keys are MIDI file paths (as strings) and values are booleans indicating validity.
    """
    valid_midi_dict = {}
    for midi_path in list(Path(path).glob("**/*.mid")) + list(Path(path).glob("**/*.midi")):
        try:
            midi_data = pretty_midi.PrettyMIDI(str(midi_path))
            valid = any(len(instrument.notes) >= MIN_NB_NOTES for instrument in midi_data.instruments)
            valid_midi_dict[str(midi_path)] = valid
        except Exception as e:
            print(f"Error loading {midi_path}: {e}")
            valid_midi_dict[str(midi_path)] = False  # Consider invalid if there was an error loading it

    return valid_midi_dict

In [7]:
def load_dataset(path: str) -> List:
  """
  dataset: The directory name of the dataset
  path: The path to the dataset
  """
  dataset = path.split("/")[-1]
  parent_directory = Path(path).parent  # This gets the parent directory of the dataset
  print("=====> Parent directory:", parent_directory)
  new_directory_name = f"{dataset}-chunked"
  new_directory = parent_directory / new_directory_name  # This creates the new path correctly

  print("=====> Created new directory:", new_directory)
  new_directory.mkdir(parents=True, exist_ok=True)

  valid_midi_dict = load_valid_midi_files(path)
  valid_midi_paths = [Path(key) for key, value in valid_midi_dict.items() if value]
  # valid_midi_paths = list(Path(path).glob("**/*.mid")) + list(Path(path).glob("**/*.midi"))

  return new_directory, valid_midi_paths

In [8]:
midi_files = load_valid_midi_files(path)
print("Total midi files: ", len(midi_files), end='\n')
print("Valid midi files =======>")
for key, value in midi_files.items():
  if value:
    print(key)
  else:
    print(f"Not valid midi: {key}")

Total midi files:  1
Valid midi files =======>
test\prompt\0_prompt.wav.mid


In [9]:
new_directory, midi_paths = load_dataset(path)

=====> Parent directory: test
=====> Created new directory: test\prompt-chunked


In [10]:
len(midi_paths)

1

# Create chunks and store them in the new folder

In [11]:
def create_chunks(path, new_directory: str, midi_paths: List[str]):
  """
  midi_paths: The paths to the MIDI files to be chunked
  """
  for i, midi_path in enumerate(tqdm(midi_paths, desc="CHUNKING MIDIS")):
    try:

      # Determine the output directory for the files
      relative_path = midi_path.relative_to(path)
      output_directory = new_directory / relative_path.parent
      output_directory.mkdir(parents=True, exist_ok=True)

      # Check if chunks are already created
      chunk_paths = list(output_directory.glob(f"{midi_path.stem}_*.mid"))
      if len(chunk_paths) > 0:
        print(f"\n=====> Chunks already created for {midi_path}, skipping")
        continue

      # Loads MIDI, merges and saves it
      midi = MidiFile(str(midi_path))
      ticks_per_cut = MAX_NB_BAR * midi.ticks_per_beat * 4
      nb_cuts = ceil(midi.max_tick / ticks_per_cut)
      if nb_cuts < 2:
        print(f"=====> Less than two chunks for {midi_path}, skipping....")
        continue

      print(f"Processing {midi_path}")
      midis = [deepcopy(midi) for _ in range(nb_cuts)]

      for j, track in enumerate(midi.instruments):
        track.notes.sort(key = lambda x: x.start)
        for midi_short in midis:
          midi_short.instruments[j].notes = []
        for note in track.notes:
          cut_id = note.start // ticks_per_cut
          note_copy = deepcopy(note)
          note_copy.start -= cut_id * ticks_per_cut
          note_copy.end -= cut_id * ticks_per_cut
          midis[cut_id].instruments[j].notes.append(note_copy)

      # Saving the Midis
      for j, midi_short in enumerate(midis):
        if sum(len(track.notes) for track in midi_short.instruments) < MIN_NB_NOTES:
          continue
        midi_short.dump(output_directory / f"{midi_path.stem}_{j}.mid")

    except Exception as e:
      print(f"\nError creating output directory: {e}")

In [12]:
create_chunks(path, new_directory, midi_paths)

CHUNKING MIDIS: 100%|██████████| 1/1 [00:00<00:00, 250.02it/s]

=====> Less than two chunks for test\prompt\0_prompt.wav.mid, skipping....


* The total english songs dataset had around 330 songs out of which 7 songs were corrupted. Skipping those songs would make around 323 songs for the complete dataset.
* Only 50 hindi songs were choosen for the training due to un-availability of songs. future works would be gathering more songs for training
